In [1]:
import torch
import torch.nn as nn

import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [2]:
import visdom

vis = visdom.Visdom()
vis.close(env="main")

Setting up a new session...


''

In [3]:
def loss_tracker(loss_plot, loss_value, num):
    vis.line(X=num,
             Y=loss_value,
             win = loss_plot,
             update ='append')

In [4]:
device = "cuda" if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                       download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=512,
                                         shuffle=True, num_workers=4)

testset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                         shuffle=True, num_workers=4)

classes = ('plane', 'car', 'bird', 'cat',
          'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [6]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

def imshow(img):
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
    
dataiter = iter(trainloader)
images, labels = dataiter.next()
vis.images(images/2+0.5)

print(''.join('%5s' % classes[labels[j]] for j in range(4)))

 shiphorse bird frog


In [8]:
import torchvision.models.vgg as vgg

In [9]:
cfg = [32,32,'M',64,64,128,128,128,'M',256,256,256,512,512,512,'M'] # 13+3 = vgg16

In [26]:
class VGG(nn.Module):

    def __init__(
        self,
        features,
        num_classes: int = 1000,
        init_weights: bool = True
    ) -> None:
        super(VGG, self).__init__()
        self.features = features
#         self.avgpool = nn.AdaptiveAvgPool2d((7, 7)) 7*7보다 작으면 안쓰는게 좋지
        self.classifier = nn.Sequential(
            nn.Linear(512 * 4 * 4, 4096), #이미지 사이즈가 다를땐 여기를 수정해줘야함
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
#       x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self) -> None:
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

In [27]:
vgg16=VGG(vgg.make_layers(cfg), 10, True).to(device)

In [28]:
a=torch.Tensor(1,3,32,32).to(device)
out = vgg16(a)
print(out)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], grad_fn=<AddmmBackward>)


In [30]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(vgg16.parameters(), lr = 0.005, momentum=0.9)

lr_sche = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)

In [32]:
loss_plt = vis.line(Y=torch.Tensor(1).zero_(), opts=dict(title="loss_tracker", legend=['loss'], showlegend=True))

In [ ]:
print(len(trainloader))
epochs = 50

for epoch in range(epochs):
    running_loss = 0.0
    optimizer.step()
    
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = vgg16(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        lr_sche.step()
        
        running_loss += loss.item()
        if i % 30 == 29:
            loss_tracker(loss_plt, torch.Tensor([running_loss/30]), torch.Tensor([i + epoch*len(trainloader)]))
            print('[%d, %5d] loss: %.3f' %
                     (epoch + 1, i + 1, running_loss / 30))
            running_loss = 0.0
            
print('Finish Training')

98
[1,    30] loss: 2.303
[1,    60] loss: 2.303
[1,    90] loss: 2.303
[2,    30] loss: 2.303
[2,    60] loss: 2.303
[2,    90] loss: 2.303


In [ ]:
dataiter = iter(testloader)
images, labels = dataliter.next()

imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
outputs = vgg16(images.to(device))

In [ ]:
_, predicted = torch.max(outputs, 1)
print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                             for j in range(4)))

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = vgg16(images)
        
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        
        correct += (predicted == labels).sum().item()
        
print('Accuracy of the network on the 10000 test images: %d %%' % (
   100 * correct / total))